Выбранные произведения

М. Горький – "Мать" (gorky_mat.txt): насыщенный эмоциональный текст, подходит для анализа эпитетов и наречий.

А. Куприн – "Яма" (kuprin_yama.txt): реалистичное описание, богатое на прилагательные и наречия.

И. Бунин – "Деревня" (bunin_derevnya.txt): короткий роман с выразительным языком.

Н. Лесков – "Левша" (leskov_levsha.txt): насыщенный художественный язык, множество прилагательных.

А. Чехов – "Палата №6" (chekhov_palata_no_6.txt): компактное произведение, подходящее для быстрой обработки.

И. Тургенев – "Отцы и дети" (turgenev_otcy_i_deti.txt): классический роман с разнообразным словарным запасом.

Ф. Достоевский – "Идиот" (dostoevsky_idiot.txt): глубокий психологический роман, полезен для анализа сложных конструкций.

Л. Толстой – "Анна Каренина" (tolstoy_anna_karenina.txt): обширный роман, можно использовать выборочно для анализа.

In [1]:
FILE_LIST = [
    "Bulgakov_Master.txt",
    "Bulgakov_ZapiskiYonogoVracha.txt",
    "Gorky_ZyznKlimaSamgina1.txt",
    "Gorky_ZyznKlimaSamgina2.txt",
    "Gorky_ZyznKlimaSamgina3.txt",
    "Gorky_ZyznKlimaSamgina4.txt",
    "Nabokov_Dar_1938.txt",
    "Nabokov_Podvig_1932.txt",
    "Nabokov_Zashchita_1930.txt",
    "Sologub_MelkijBes.txt"
]

In [2]:
import re
import os

RAW_DIR = "RussianNovels/corpus"
CLEAN_DIR = "data/clean"
os.makedirs(CLEAN_DIR, exist_ok=True)

def clean_text(text: str) -> str:
    text = re.sub(r'\ufeff', '', text)
    text = re.sub(r'\r\n?', '\n', text)
    text = re.sub(r'[ \t]+', ' ', text) 
    text = re.sub(r'\n{2,}', '\n', text)
    text = text.lower() 
    return text.strip()

for filename in os.listdir(RAW_DIR):
    if filename.endswith(".txt"):
        path = os.path.join(RAW_DIR, filename)
        with open(path, encoding='utf-8') as f:
            raw = f.read()
        cleaned = clean_text(raw)
        out_path = os.path.join(CLEAN_DIR, filename)
        with open(out_path, 'w', encoding='utf-8') as f:
            f.write(cleaned)


In [3]:
import spacy

print("Загружаем модель spaCy...")
nlp = spacy.load("ru_core_news_lg")
print("Модель загружена.")


Загружаем модель spaCy...
Модель загружена.


In [4]:
import re

def has_descriptive_language(sentence):
    adj_adv_count = sum(1 for token in sentence if token.pos_ in ['ADJ', 'ADV'])
    return adj_adv_count >= 2


In [5]:
import re

def is_dialogue(sentence):
    text = sentence.text.strip()
    return text.startswith("—") or text.startswith("--") or re.match(r'["«].+["»]', text)

In [6]:
def is_meaningful(sent, min_len=12, max_len=18):
    content_words = [token for token in sent if token.pos_ in {"NOUN", "PROPN", "VERB", "AUX", "ADJ", "ADV"}]
    return min_len <= len(content_words) <= max_len

def preprocess_text(text: str):
    lines = text.splitlines()
    cleaned_lines = []
    buffer = ""

    for line in lines:
        line = line.strip()
        if not line:
            continue
        if line.startswith("--"):
            if buffer:
                cleaned_lines.append(buffer.strip())
                buffer = ""
            cleaned_lines.append(line)
        else:
            if line.endswith("...") or line.endswith(".") or line.endswith("?") or line.endswith("!"):
                buffer += " " + line
                cleaned_lines.append(buffer.strip())
                buffer = ""
            else:
                buffer += " " + line

    if buffer:
        cleaned_lines.append(buffer.strip())

    return cleaned_lines

In [7]:
stopwords = nlp.Defaults.stop_words
def remove_stopwords_from_spacy_doc(doc, stopwords):
    tokens = [token.text for token in doc if token.text.lower() not in stopwords and not token.is_punct]
    return " ".join(tokens)

In [8]:
import os
import spacy

def segment_and_filter(
    path: str,
    min_len: int = 12,
    max_len: int = 18,
    out_dir: str = "data/filtered"
):
    print(f"Чтение файла: {path}")
    with open(path, encoding='utf-8') as f:
        raw_text = f.read()

    print("Предобработка строк...")
    lines = preprocess_text(raw_text)

    sentences = []
    print("Анализ предложений spaCy и фильтрация...")
    for line in lines:
        doc = nlp(line)
        for sent in doc.sents:
            cleaned_text = remove_stopwords_from_spacy_doc(sent, stopwords)
            if not cleaned_text.strip():
                continue
            cleaned_doc = nlp(cleaned_text)
    
            if is_dialogue(cleaned_doc):
                continue
            if not is_meaningful(cleaned_doc, min_len, max_len):
                continue
            if not has_descriptive_language(cleaned_doc):
                continue

        sentences.append(cleaned_text)

    print(f"Подходящих предложений найдено: {len(sentences)}")
    os.makedirs(out_dir, exist_ok=True)

    filename = os.path.basename(path)
    out_path = os.path.join(out_dir, filename)

    with open(out_path, 'w', encoding='utf-8') as f_out:
        for s in sentences:
            f_out.write(s + '\n')

    print(f"Результат сохранён в: {out_path}")
    return sentences

In [9]:
TEXT_DIR = "data/clean"
OUT_DIR = "data/filtered"

for filename in FILE_LIST:
    path = os.path.join(TEXT_DIR, filename)
    segment_and_filter(path, out_dir=OUT_DIR)

Чтение файла: data/clean\Bulgakov_Master.txt
Предобработка строк...
Анализ предложений spaCy и фильтрация...
Подходящих предложений найдено: 3454
Результат сохранён в: data/filtered\Bulgakov_Master.txt
Чтение файла: data/clean\Bulgakov_ZapiskiYonogoVracha.txt
Предобработка строк...
Анализ предложений spaCy и фильтрация...
Подходящих предложений найдено: 1417
Результат сохранён в: data/filtered\Bulgakov_ZapiskiYonogoVracha.txt
Чтение файла: data/clean\Gorky_ZyznKlimaSamgina1.txt
Предобработка строк...
Анализ предложений spaCy и фильтрация...
Подходящих предложений найдено: 4394
Результат сохранён в: data/filtered\Gorky_ZyznKlimaSamgina1.txt
Чтение файла: data/clean\Gorky_ZyznKlimaSamgina2.txt
Предобработка строк...
Анализ предложений spaCy и фильтрация...
Подходящих предложений найдено: 5339
Результат сохранён в: data/filtered\Gorky_ZyznKlimaSamgina2.txt
Чтение файла: data/clean\Gorky_ZyznKlimaSamgina3.txt
Предобработка строк...
Анализ предложений spaCy и фильтрация...
Подходящих предло

In [10]:
TEXT_DIR = "data/clean"
OUT_DIR = "data/filtered"

for filename in os.listdir(TEXT_DIR):
    if filename.endswith(".txt"):
        path = os.path.join(TEXT_DIR, filename)
        segment_and_filter(path, out_dir=OUT_DIR)

Чтение файла: data/clean\Bulgakov_BelayaGvardiya.txt
Предобработка строк...
Анализ предложений spaCy и фильтрация...
Подходящих предложений найдено: 2709
Результат сохранён в: data/filtered\Bulgakov_BelayaGvardiya.txt
Чтение файла: data/clean\Bulgakov_Diavoliada.txt
Предобработка строк...
Анализ предложений spaCy и фильтрация...
Подходящих предложений найдено: 408
Результат сохранён в: data/filtered\Bulgakov_Diavoliada.txt
Чтение файла: data/clean\Bulgakov_Master.txt
Предобработка строк...
Анализ предложений spaCy и фильтрация...
Подходящих предложений найдено: 3454
Результат сохранён в: data/filtered\Bulgakov_Master.txt
Чтение файла: data/clean\Bulgakov_RokovyeYaytsa.txt
Предобработка строк...
Анализ предложений spaCy и фильтрация...
Подходящих предложений найдено: 688
Результат сохранён в: data/filtered\Bulgakov_RokovyeYaytsa.txt
Чтение файла: data/clean\Bulgakov_TeatralnyjRoman.txt
Предобработка строк...
Анализ предложений spaCy и фильтрация...
Подходящих предложений найдено: 1731
Р